# TF-IDF (Term Frecuency - Inverse Document Frecuency)

TF-IDF (Frecuencia de Terminos - Frecuecia de Documentos Inverso) es un método de vectorización de características ampliamente usado en minería de texto para reflejar la importancia de un termino dentro de un documento en el cuerpo (todos los documentos).

$$ IDF(t, D) = \log \frac{|D| + 1}{DF(t, D) + 1} $$

- t es el termino
- d es un documento
- D es el cuerpo de documentos

$$ TF-IDF(t, d, D) = TF(t, d) \cdot IDF(t, D). $$

- TF(t,d) es la frecuencia del termino t en el documento d

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('tfidf').getOrCreate()

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [0]:
df_documentos = spark.createDataFrame([
    (0.0,"Hola esta es una prueba con Spark"),
    (1.0,"Java está incluida para su uso en Spark"),
    (2.0,"Sigmoide es una función de activación")
],["etiqueta","oracion"])

In [0]:
df_documentos.show(truncate=False)

+--------+---------------------------------------+
|etiqueta|oracion                                |
+--------+---------------------------------------+
|0.0     |Hola esta es una prueba con Spark      |
|1.0     |Java está incluida para su uso en Spark|
|2.0     |Sigmoide es una función de activación  |
+--------+---------------------------------------+



In [0]:
toquenizador = Tokenizer(inputCol="oracion",outputCol="palabras")

In [0]:
df_palabras = toquenizador.transform(df_documentos)

In [0]:
df_palabras.show(truncate=False)

+--------+---------------------------------------+------------------------------------------------+
|etiqueta|oracion                                |palabras                                        |
+--------+---------------------------------------+------------------------------------------------+
|0.0     |Hola esta es una prueba con Spark      |[hola, esta, es, una, prueba, con, spark]       |
|1.0     |Java está incluida para su uso en Spark|[java, está, incluida, para, su, uso, en, spark]|
|2.0     |Sigmoide es una función de activación  |[sigmoide, es, una, función, de, activación]    |
+--------+---------------------------------------+------------------------------------------------+



In [0]:
ft_hashing = HashingTF(inputCol="palabras",outputCol="caracteristicas01",numFeatures=32)

In [0]:
datos_caracterizados = ft_hashing.transform(df_palabras)

In [0]:
datos_caracterizados.show()

+--------+--------------------+--------------------+--------------------+
|etiqueta|             oracion|            palabras|   caracteristicas01|
+--------+--------------------+--------------------+--------------------+
|     0.0|Hola esta es una ...|[hola, esta, es, ...|(32,[1,5,7,16,18,...|
|     1.0|Java está incluid...|[java, está, incl...|(32,[6,9,12,20,22...|
|     2.0|Sigmoide es una f...|[sigmoide, es, un...|(32,[1,4,9,16,18,...|
+--------+--------------------+--------------------+--------------------+



In [0]:
fdi = IDF(inputCol='caracteristicas01',outputCol='caracteristicas')

In [0]:
modelo_fdi = fdi.fit(datos_caracterizados)

In [0]:
datos_transformados = modelo_fdi.transform(datos_caracterizados)

In [0]:
datos_transformados.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|etiqueta|             oracion|            palabras|   caracteristicas01|     caracteristicas|
+--------+--------------------+--------------------+--------------------+--------------------+
|     0.0|Hola esta es una ...|[hola, esta, es, ...|(32,[1,5,7,16,18,...|(32,[1,5,7,16,18,...|
|     1.0|Java está incluid...|[java, está, incl...|(32,[6,9,12,20,22...|(32,[6,9,12,20,22...|
|     2.0|Sigmoide es una f...|[sigmoide, es, un...|(32,[1,4,9,16,18,...|(32,[1,4,9,16,18,...|
+--------+--------------------+--------------------+--------------------+--------------------+



In [0]:
datos_transformados.select('etiqueta','caracteristicas').show(truncate=False)

+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|etiqueta|caracteristicas                                                                                                                                                    |
+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0     |(32,[1,5,7,16,18,19,22],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.28768207245178085,0.28768207245178085,0.6931471805599453,0.28768207245178085])|
|1.0     |(32,[6,9,12,20,22,28,31],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.28768207245178085,0.28768207245178085,0.6931471805599453,1.3862943611198906])|
|2.0     |(32,[1,4,9,16,18,20],[0.28768207245178085,0.6931471805599453,0.28768207245178085,0.28768207245178085,0.287682072451

# CountVectorizer y CountVectorizerModel

CountVectorizer y CountVectorizerModel permiten convertir una colección de documentos de texto a vectores de conteo de tokens.

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0,"a b c".split(" ")),
    (1,"a b b c a".split(" "))
],["id","palabras"])

In [0]:
df.show(truncate=False)

+---+---------------+
|id |palabras       |
+---+---------------+
|0  |[a, b, c]      |
|1  |[a, b, b, c, a]|
+---+---------------+



In [0]:
vc = CountVectorizer(inputCol="palabras",
                     outputCol="caracteristicas",
                     vocabSize=3,
                     minDF=2.0)

In [0]:
modelo = vc.fit(df)

In [0]:
resultado = modelo.transform(df)

In [0]:
resultado.show(truncate=False)

+---+---------------+-------------------------+
|id |palabras       |caracteristicas          |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

